In [1]:
!git clone https://github.com/alexanderkhoe/DDD.git

Cloning into 'DDD'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 15 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (15/15), 20.96 KiB | 4.19 MiB/s, done.


In [2]:
%cd /content/DDD

/content/DDD


In [3]:
!pip install SALib
!pip install -U pymoo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.8/778.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 20.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for grapheme: filename=grapheme-0.6.0-py3-none-any.whl size=210078 sha256=c87053d7695379cf7e79b36c36ebbd270c36a8297524f7591b984631588ec077
  Stored in directory: /root/.cache/pip/wheels/01/e1/49/37e6bde9886439057450c494a79b0bef8bbe897a54aebfc757
Successfully built grapheme


In [4]:
from DDD.cdm import CausalDecisionModel
from DDD.element import Lever, Outcome, Intermediate, External

lever1 = Lever("Marketing Spend", 500.00, [0.00, 2000.00])
lever2 = Lever("Sales Price", 8.25, [0.04, 15.00])
lever3 = Lever("Production Order Size", 85000, [0, 200000])



In [5]:
external1 = External("Market Size", 1000, [100, 10000])


In [6]:
def f(x):
  return x

itm1 = Intermediate("Demand", ["Sales Price"]).fit(f)
itm2 = Intermediate("Uplift", ["Marketing Spend"]).fit(lambda x: x)
itm3 = Intermediate("Units Sold", ["Market Size", "Uplift", "Demand"]).fit(lambda x, y, z: x*y*z)
itm4 = Intermediate("Revenue", ["Units Sold", "Sales Price"]).fit(lambda x, y: x * y)

itm5 = Intermediate("Unit Cost", ["Production Order Size"]).fit(lambda x: x)
itm6 = Intermediate("Production Cost", ["Unit Cost", "Production Order Size"]).fit(lambda x, y: x * y)
itm7 = Intermediate("Total Cost", ["Marketing Spend", "Production Cost"]).fit(lambda x, y: x + y)


In [7]:
outcome = Outcome("Profit", ["Revenue", "Total Cost"]).fit(lambda x, y: x - y)


In [8]:
model = CausalDecisionModel([lever1, lever2, lever3])
model.add_element(external1)
model.add_elements([itm1, itm2, itm3, itm4, itm5, itm6, itm7])
model.add_element(outcome)

In [9]:
print(model.validate())
print(model.element("Total Cost"))
print(model.call("Profit")([1, 2, 3]))
print(model.sensitivity("Profit"))
x, y, z = model.multi_optimize(["Profit"])
print(x, y, z)
print(model.fidelity)

print(model.optimize("Profit"))
external1.value = 3000
print(model.call("Profit")([1, 2, 3]))
print(model.optimize("Profit"))
print(model.call("Profit")([1, 2, 3]))

True
Intermediate(Total Cost)
3990
(array([7.47332982e-05, 2.98569822e-05, 9.99972453e-01]), array([2.37697106e-05, 4.20977393e-05, 9.99981792e-01]))
1999.854858858809 14.994840729777122 12.159563110110184
[1, 1, 1, 1, 1, 1, 1, 1]
1.0
[1999.95846543   14.99975375   24.24529037]
11990
[1999.97712782   14.99928067    8.75145792]
11990


In [ ]:
z

5.735768680018168